In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-22 23:28:32,772 utils 400 [INFO]    [logger_func] start 


In [44]:
#========================================================================
# Base Model Load
base = utils.read_df_pkl('../input/base_no_out_clf.gz')[[key, 'first_active_month', 'hist_purchase_month_max', target, 'clf_pred', 'no_out_flg']].set_index(key)
# Alijs type
df_type = pd.read_csv('../input/card_ids_grouping.csv').set_index(key)
base = base.join(df_type)
#========================================================================

min_thres = 0.01
pred = 'prediction'
with_outlier = 'base_pred'
#========================================================================
# Remove Out

# Low Outlier Ratio
type0_flg0_lower = ( (base['type']==0) & (base['no_out_flg']==0) ) & (base['clf_pred']<min_thres)
type0_flg1_lower = ( (base['type']==0) & (base['no_out_flg']==1) ) & (base['clf_pred']<min_thres)
type1_flg0_lower = ( (base['type']==1) & (base['no_out_flg']==0) ) & (base['clf_pred']<min_thres)
type1_flg1_lower = ( (base['type']==1) & (base['no_out_flg']==1) ) & (base['clf_pred']<min_thres)
type2_flg0_lower = ( (base['type']==2) & (base['no_out_flg']==0) ) & (base['clf_pred']<min_thres)
type2_flg1_lower = ( (base['type']==2) & (base['no_out_flg']==1) ) & (base['clf_pred']<min_thres)

#         # High Outlier Ratio
type0_flg0_higher = ( (base['type']==0) & (base['no_out_flg']==0) ) & (base['clf_pred']>=min_thres)
type0_flg1_higher = ( (base['type']==0) & (base['no_out_flg']==1) ) & (base['clf_pred']>=min_thres)
type1_flg0_higher = ( (base['type']==1) & (base['no_out_flg']==0) ) & (base['clf_pred']>=min_thres)
type1_flg1_higher = ( (base['type']==1) & (base['no_out_flg']==1) ) & (base['clf_pred']>=min_thres)
type2_flg0_higher = ( (base['type']==2) & (base['no_out_flg']==0) ) & (base['clf_pred']>=min_thres)
type2_flg1_higher = ( (base['type']==2) & (base['no_out_flg']==1) ) & (base['clf_pred']>=min_thres)
type10 = (base['type']==10)

lower_clf = (base['clf_pred']<0.01)
flg1 = (base['no_out_flg']==1)

100%|██████████| 1/1 [00:00<00:00,  4.13it/s]


In [45]:
pred_col = 'prediction'
base_pred = pd.read_csv('../ensemble/good_submit_ensemble/0222_2031_alijs_stack_submit_pp_02222031_CV3.6059_LB3.659.csv').set_index(key)
base[pred_col] = base_pred

In [46]:
val_col = 'valid_pred'
valid_pred = pd.read_csv('../ensemble/good_submit_ensemble/aljis_stack_submit_train_02202319_CV3.6003_LB3.663.csv').set_index(key)
base[val_col] = valid_pred
base.loc[type1_flg0_lower  , pred_col] = base.loc[type1_flg0_lower  , val_col]
base.head(50)

,first_active_month,hist_purchase_month_max,target,clf_pred,no_out_flg,type,prediction,valid_pred
card_id,,,,,,,,
C_ID_92a2005557,2017-06-01,2018-03-01,-0.820283,0.000444,1.0,2,-0.319724,-0.306923
C_ID_3d0044924f,2017-01-01,2018-02-01,0.392913,0.007831,0.0,0,-0.022252,-0.021068
C_ID_d639edf6cd,2016-08-01,2018-03-01,0.688056,0.004074,0.0,2,0.773737,0.762709
C_ID_186d6a6901,2017-09-01,2018-03-01,0.142495,0.000797,0.0,2,0.203239,0.199781
C_ID_cdbd2c0db2,2017-11-01,2018-03-01,-0.159749,0.000251,1.0,10,-0.289072,-0.260867
C_ID_0894217f2f,2016-09-01,2017-08-01,0.871585,0.043221,0.0,0,-1.893529,-1.956644
C_ID_7e63323c00,2016-12-01,2018-03-01,0.230129,0.000891,0.0,2,-0.223559,-0.223427
C_ID_dfa21fc124,2017-09-01,2018-01-01,2.135850,0.001918,1.0,1,-0.040444,-0.020724
C_ID_fe0fdac8ea,2017-08-01,2018-01-01,-0.065406,0.007651,0.0,2,0.481522,0.455255


In [48]:
#========================================================================
# Make Submission
#========================================================================
# check cv
train = base[~base[target].isnull()]
y_pred = train[pred_col]
y_train = train[target]
score = np.sqrt(mean_squared_error(y_train, y_pred))
print(score)
#========================================================================

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred_col]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit_path = f'{start_time[4:12]}_submit_valid_CV{str(score)[:8]}_LB'
submit.to_csv(f'../submit/{submit_path}')
utils.to_pkl_gzip(obj=base.reset_index()[[key, target, 'prediction']], path=f"../stack/{submit_path.replace('submit', 'stack_submit_log')}")
display(submit.head())
#========================================================================

3.605799438954167


,target
card_id,
C_ID_0ab67a22ab,-1.309191
C_ID_130fd0cbdd,-0.144377
C_ID_b709037bc5,-0.794374
C_ID_d27d835a9f,-0.077131
C_ID_2b5e3df5c2,-1.381594
